In [3]:
import tensorflow as tf
import foolbox as fb
import foolbox.attacks as fa
import numpy as np
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
GPU_NUM=3
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:'+str(GPU_NUM)
        print("using gpu %s: " % device)
    else:
        device = 'cpu'
        print("using cpu")
    return device
device=get_device()

with tf.Session() as session:
  new_saver = tf.train.import_meta_graph('./CNN_tf_models/natural/checkpoint-24900.meta)
  new_saver.restore(session, tf.train.latest_checkpoint('./CNN_tf_models/natural'))
#   print(sess.run('w1:0'))


batch_size=5
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

testset = MNIST('./data', transform=img_transform, download=True, train=False)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

for batch in testloader:
    images, labels = batch
    images=images.to(device)
    labels=labels.to(device)
    imagesVectors =torch.zeros(images.shape[0],images.shape[2]*images.shape[3],device=device)
    # change image in images to 784 rows vector
    imagesVectors = images.view(images.size(0), -1)
    break

In [ ]:
fmodel = fb.TensorFlowModel(session,bounds=(-1,1),device=device)

clean = fb.utils.accuracy(fmodel,imagesVectors, labels)
attack = fb.attacks.LinfFastGradientAttack()

raw, clipped, is_adv = attack(fmodel,imagesVectors,labels,epsilons=0.03)

np.save('pic_for_LinfFGSM.npy',torch.Tensor.cpu(clipped))
np.save('is_adv_for_LinfFGSM',torch.Tensor.cpu(is_adv))